# Pattern processing

The raw EBSD signal can be empirically evaluated as a superposition of a Kikuchi
diffraction pattern and a smooth background intensity. For pattern indexing, the
latter intensity is usually undesirable, while for [virtual backscatter electron
(VBSE) imaging](virtual_backscatter_electron_imaging.rst), this intensity
can reveal topographical, compositional or diffraction contrast. This section
details methods to enhance the Kikuchi diffraction pattern and manipulate
detector intensities in patterns in an [EBSD](reference.rst#kikuchipy.signals.EBSD)
object.

Most of the methods operating on EBSD objects use functions that operate on the
individual patterns (`numpy.ndarray`). These single pattern functions are
available in the [kikuchipy.pattern](reference.rst#pattern) module.

Let's import necessary libraries and read the Nickel EBSD data set:

In [ ]:
#%matplotlib qt5
%matplotlib inline

import os
import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp


datadir = "../../kikuchipy_data/ni/nordif"
s = kp.load(os.path.join(datadir, "Pattern.dat"))

Almost all methods operate inplace (indicated in their docstrings), meaning it
overwrites the patterns in the EBSD object. If instead we want a new object,
we create a [deepcopy](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.signal.html#hyperspy.signal.BaseSignal.deepcopy)
of the original object and perform our operations on this. To demonstrate this,
we create a new EBSD signal from a small part of the original signal:

In [ ]:
x0, x1 = (156, 159)
y0, y1 = (77, 80)
s2 = s.inav[x0:x1, y0:y1].deepcopy()
np.may_share_memory(s.data, s2.data)

## Rescale intensity

Vendors usually write patterns to file with 8 (``uint8``) or 16 (``uint16``) bit
integer depth, holding [0, 2$^8$] or [0, 2$^{16}$] gray levels, respectively. To
avoid losing intensity information when processing, we often change data types
to e.g. 32 bit floating point (``float32``). However, only changing the data
type with [change_dtype](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.signal.html#hyperspy.signal.BaseSignal.change_dtype) does not
rescale pattern intensities, leading to patterns not using the full available
data type range:

In [ ]:
print(s2.data.dtype, s2.data.max())

In [ ]:
s2.change_dtype(np.uint16)

In [ ]:
print(s2.data.dtype, s2.data.max())

In [ ]:
s2.plot(vmax=1000)

In these cases it is convenient to rescale intensities to a desired data type
range, either keeping relative intensities between patterns in a scan or not. We
can do this for all patterns in a scan
([EBSD](reference.rst#kikuchipy.signals.EBSD) object) with [kikuchipy.signals.EBSD.rescale_intensity](reference.rst#kikuchipy.signals.EBSD.rescale_intensity):

In [ ]:
s2.rescale_intensity(relative=True)
print(s2.data.dtype, s2.data.max())

In [ ]:
s2.plot(vmax=65535)

Or, we can do it for a single pattern (`numpy.ndarray`) with
[kikuchipy.pattern.rescale_intensity](reference.rst#kikuchipy.pattern.rescale_intensity):

In [ ]:
p = s.inav[x0, y0].data
print(p.min(), p.max())
p2 = kp.pattern.rescale_intensity(p, dtype_out=np.uint16)
print(p2.min(), p2.max())

We can also stretch the pattern contrast by removing intensities outside a range
passed to `in_range` or at certain percentiles by passing percents to
`percentiles`:

In [ ]:
s3 = s.inav[x0:x1, y0:y1].deepcopy()
s3.rescale_intensity(in_range=(5, 250))
print(s3.data.min(), s3.data.max())

In [ ]:
s3.rescale_intensity(percentiles=(0.5, 99.5))
print(s3.data.min(), s3.data.max())

This can reduce the influence of outliers with exceptionally high or low
intensities, like hot or dead pixels.

In [ ]:
fig, ax = plt.subplots(figsize=(11, 4), ncols=2)
im0 = ax[0].imshow(s2.inav[0, 0].data, cmap="gray")
fig.colorbar(im0, ax=ax[0])
im1 = ax[1].imshow(s3.inav[0, 0].data, cmap="gray")
fig.colorbar(im1, ax=ax[1])